In [1]:
#!python -m pip install --upgrade pip

#!pip install tensorflow==2.3.1 

#!pip install tensorboard==2.4.1
!pip install torch  #YOLOv5 runs on top of PyTorch, so we need to import it to the notebook

In [2]:
import torch # YOLOv5 implemented using pytorch

In [3]:
from IPython.display import Image #this is to render predictions

In [4]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 14936, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 14936 (delta 12), reused 16 (delta 4), pack-reused 14908
Receiving objects: 100% (14936/14936), 14.01 MiB | 48.81 MiB/s, done.
Resolving deltas: 100% (10259/10259), done.


In [5]:
%cd yolov5

/home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5


In [6]:
!pip install -r requirements.txt

In [21]:
#unzipped data
import glob
import zipfile

for file in files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall('data/raw')

Divide the dataset in train and val folder.

In [47]:
import os
from random import choice
import shutil
import pathlib


#arrays to store file names
imgs =[]
xmls =[]

#setup dir names
trainPath = '/home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/dataset/images/train'
valPath = '/home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/dataset/images/val'
crsPath = '/home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/data/raw/ts' #dir where images and annotations stored

#setup ratio (val ratio = rest of the files in origin dir after splitting into train and test)
train_ratio = 0.8
val_ratio = 0.2


#total count of imgs
totalImgCount = len(os.listdir(crsPath))/2

#soring files to corresponding arrays
for (dirname, dirs, files) in os.walk(crsPath):
    for filename in files:
        if filename.endswith('.txt'):
            xmls.append(filename)
        else:
            imgs.append(filename)


#counting range for cycles
countForTrain = int(len(imgs)*train_ratio)
countForVal = int(len(imgs)*val_ratio)
print("training images are : ",countForTrain)
print("Validation images are : ",countForVal)


training images are :  592
Validation images are :  148


In [48]:
trainimagePath = '/home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/dataset/images/train'
trainlabelPath = '/home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/dataset/labels/train'
valimagePath = '/home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/dataset/images/val'
vallabelPath = '/home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/dataset/labels/val'
#cycle for train dir
for x in range(countForTrain):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))

    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))


    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)



#cycle for test dir   
for x in range(countForVal):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    
    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)

#rest of files will be validation files, so rename origin dir to val dir
#os.rename(crsPath, valPath)
shutil.move(crsPath, valPath)

'/home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/dataset/images/val/ts'

running yolov5

In [58]:
!python train.py --img 415 --batch 16 --epochs 30 --data dataset.yaml --weights yolov5s.pt --cache

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=415, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-59-gfdc35b1 Python-3.8.8 torch-1.7.0 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, wa

In [60]:

!python detect.py --source runs/train/exp7/a.jpg --weights runs/train/exp7/weights/best.pt

detect: weights=['runs/train/exp7/weights/best.pt'], source=runs/train/exp7/a.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-59-gfdc35b1 Python-3.8.8 torch-1.7.0 CPU

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/runs/train/exp7/a.jpg: 384x640 1 prohibitory, 1 mandatory, 66.4ms
Speed: 0.4ms pre-process, 66.4ms inference, 0.8ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp2


In [61]:
!python detect.py --source runs/train/exp7/b.jpg --weights runs/train/exp7/weights/best.pt

detect: weights=['runs/train/exp7/weights/best.pt'], source=runs/train/exp7/b.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-59-gfdc35b1 Python-3.8.8 torch-1.7.0 CPU

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /home/ubuntu/notebooks/Hari_Solomons_Development/az/yolov5/runs/train/exp7/b.jpg: 384x640 1 mandatory, 2 others, 61.4ms
Speed: 0.4ms pre-process, 61.4ms inference, 0.9ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp3
